In [1]:
import cv2
import numpy as np

# TP11

In [2]:

#BLEU
low = np.array([95,80,60])

high = np.array([115,255,150])

# ROSE
low = np.array([120,30,60])

high = np.array([160,110,255])


In [3]:
def detect_inrange(image,low,high,min,max):
    points = []

    image = cv2.blur(image,(5,5))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(image,low,high)
    elements = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
    elements = sorted(elements,key=lambda x:cv2.contourArea(x),reverse=True)

    for elem in elements:
        if cv2.contourArea(elem) < max and min < cv2.contourArea(elem):
            ((x,y),rayon) = cv2.minEnclosingCircle(elem)

            points.append([int(x),int(y),int(rayon),int(cv2.contourArea(elem))])

    return image,mask,points      

In [4]:
from math import sqrt


def pixelize(image):
    block_size=30
    array = np.ones((block_size*block_size))
    # Create a list to store the blocks
    pixels = []

    # Iterate over the matrix in steps of block_size
    for i in range(0, len(image), block_size):
        for j in range(0, len(image[i]), block_size):
            layer = ()
            # Extract the block
            for k in range(0,len(image[i][j])):
                block = image[i:i+block_size, j:j+block_size][k]
            
                # Convert the block to a 1D list
                block_list = block.flatten().tolist()
                result = sum([a*b for a,b in zip(block_list,array)]) / (block_size*block_size)
                layer = layer + (result,)
            pixels.append(layer)
    matsize = int(sqrt(len(pixels)))
    img = np.zeros(image.shape,dtype=np.uint16)
    for i in range(matsize):
        for j in range(matsize):
            img[i,j] = pixels[(i*matsize)+j]
    #print(pixels)
    return img

In [5]:
img = cv2.imread("TP.jpg",cv2.IMREAD_COLOR)

img = pixelize(img)

cv2.imshow("ixel",img)

In [6]:
cap = cv2.VideoCapture(0)
fw,fh = int(cap.get(3)),int(cap.get(4))

if not cap.isOpened():
    print("erreur")
    exit(0)

while cap.isOpened():
    ret,frame = cap.read()
    if not ret :
        print("erreur cap.reead")

    image,mask,points = detect_inrange(frame,low,high,500,4000)

    image = cv2.cvtColor(image,cv2.COLOR_HSV2BGR)
    cv2.circle(image,(100,100),20,(0,255,0),5)
    
    print(image[100,100])

    if len(points) > 0:
        point = points[0]
        cv2.circle(image,(point[0],point[1]),point[2],(255,0,0),2)
        cv2.putText(image,str(point[3]),(point[0],point[1]),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2,cv2.LINE_AA)

        image = pixelize(image)


    if mask is not None:
        cv2.imshow("mask",mask)

    cv2.imshow("cam",image)

    if cv2.waitKey(200) & 0xFF == ord("q"):
        break

out.release()
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 